![](https://img.kaikeba.com/web/kkb_index/img_index_logo.png)

# 人工智能基础课机器学习 第二节课作业 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [2]:
import pandas as pd

# 1. 从fundemantals.csv开始！

fundemantals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


In [131]:
df_base = pd.read_csv(r'./nyse/fundamentals.csv')
df_base['Period Ending'] = pd.to_datetime(df_base['Period Ending'])

df_base['year'] =pd.to_datetime(df_base['Period Ending']).dt.year.astype(str)


In [121]:
df1 = df_base[(df_base['Period Ending'] > '2015-01-01') & (df_base['Period Ending'] <= '2015-12-31')]
df1['Net Income'].mean()


1560252997.752809

In [122]:
df1[['Net Income']].apply(lambda x: x.max()-x.min())

Net Income    76922000000
dtype: int64

In [123]:
f = lambda x: x[1]/x[2]
df2 = df_base[(df_base['Period Ending'] > '2016-01-01') & (df_base['Period Ending'] <= '2016-12-31')][['Ticker Symbol','Fixed Assets','Total Assets']]
df2['RATE'] = df2.apply(f,axis =1)
df2['RATE'].mean()


0.2442857532840247

In [124]:
df2[['Ticker Symbol','RATE']].min()['Ticker Symbol']

'AAP'

# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


In [125]:
df_secu = pd.read_csv(r'./nyse/securities.csv')
df1 = df_secu.fillna('9999/12/31')
df1.groupby('Address of Headquarters')['Ticker symbol','Date first added'].min()

df2 = df_secu.fillna('1900/01/01')
df2.groupby('Address of Headquarters')['Ticker symbol','Date first added'].max()

,Ticker symbol,Date first added
Address of Headquarters,,
"Akron, Ohio",GT,1900/01/01
"Allentown, Pennsylvania",PPL,1985/4/30
"Amsterdam, Netherlands",MYL,1900/01/01
"Amsterdam, New York",MHK,2013/12/23
"Arlington, Virginia",AES,1900/01/01
...,...,...
"Woburn, Massachusetts",SWKS,2015/3/12
"Woonsocket, Rhode Island",CVS,1900/01/01
"Worsley, UK",PNR,2012/10/1


# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）

1、fundemantals 和 securities 合并，通过Ticker symbol	关联合并

In [157]:
df = pd.merge(df_base,df_secu,left_on='Ticker Symbol', right_on='Ticker symbol', how='right')
df = df[(df['year']>='2013') & (df['year']<='2016') ]
df3 = df.groupby(['Address of Headquarters'])[['Research and Development']].sum()
df3

,Research and Development
Address of Headquarters,
"Akron, Ohio",0.000000e+00
"Allentown, Pennsylvania",5.426000e+08
"Amsterdam, Netherlands",1.253700e+09
"Amsterdam, New York",0.000000e+00
"Arlington, Virginia[3]",0.000000e+00
...,...
"Woburn, Massachusetts",1.094100e+09
"Woonsocket, Rhode Island",0.000000e+00
"Worsley, UK",3.597000e+08


In [187]:
df5 = df.groupby(['Address of Headquarters','Ticker Symbol'])['Research and Development'].sum()
df5.groupby('Address of Headquarters').head(3)

Address of Headquarters   Ticker Symbol
Akron, Ohio               FE               0.000000e+00
                          GT               0.000000e+00
Allentown, Pennsylvania   APD              5.426000e+08
                          PPL              0.000000e+00
Amsterdam, Netherlands    MYL              1.253700e+09
                                               ...     
Woburn, Massachusetts     SWKS             1.094100e+09
Woonsocket, Rhode Island  CVS              0.000000e+00
Worsley, UK               PNR              3.597000e+08
York, Pennsylvania        XRAY             0.000000e+00
Zurich, Switzerland       CB               0.000000e+00
Name: Research and Development, Length: 354, dtype: float64

# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？

In [ ]:
money = 10000
for date,symbol,low,high in df['Ticker Symbol']:
    num = money/low
    money =money +(high - low)* num
print(symbol 年收益为 money)

#   美国股票是当天可以卖出，如果当天不卖出，就没有思路了